### Import required libraries

In [79]:
import re
import PyPDF2
import random
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import pathlib
import docx 
import sys
import csv
import io
nltk.download('wordnet')
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Goose\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Convert all text to lower case

In [80]:
def to_lower(text):
    for i,e in enumerate(text):
        text[i]=e.lower()
    return text

### Eliminate symbols

In [81]:
def fit_text(text):
    li=list(text.split(" "))
    li=[s.replace("\n", " ") for s in li]
    li=[s.replace(".", " ") for s in li]
    li=[s.replace("  ", " ") for s in li]
    li=[s.replace("-", " ") for s in li]
    li=[s.replace("\\", " ") for s in li]
    li=[s.replace("//", " ") for s in li]
    li=[s.replace("(", " ") for s in li]
    li=[s.replace(")", " ") for s in li]
    li=[s.replace("{", " ") for s in li]
    li=[s.replace("}", " ") for s in li]
    li=[s.replace(";", " ") for s in li]
    l=" ".join(li)
    li=l.split(" ")
    if '' in li:
        li.remove('')
    li = list(filter(None,li))
    return to_lower(li)


### Convert a .txt file into a list of words
(utility function for get_file_text() )

In [82]:
def convert_text(filename):
    textfileobj=open(filename+'.txt','r', errors="ignore")
    text=textfileobj.read()
    li=fit_text(text)
    return text
    

### Convert a .pdf file into a list of words
(utility function for get_file_text() )

In [83]:
def convert_pdf(filename):
    pdffileobj=open(filename+'.pdf','rb')
    pdfreader=PyPDF2.PdfFileReader(pdffileobj)
    pageobj=pdfreader.getPage(0)
    text=pageobj.extractText()
    li=fit_text(text)
    return text

### Convert a .docx file into a list of words
(utility function for get_file_text() )

In [84]:
def convert_docx(filename):
    text_list=[]
    doc = docx.Document(filename+".docx")
    for i in doc.paragraphs:
        text_list.append(i.text)
        text_list=" ".join(text_list)
        text_list=fit_text(text_list)
    return text_list

### Get all the synonyms of a word
(utility function for find_synonym() )

In [85]:
def get_synonyms(word):
    synonyms=list()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonyms.append(l.name())
    return synonyms 

### Checks if any synonym of the given word is in the given dictionary and returns the count of matched words
(Utility function for find_similarity_percentages)

In [86]:
def find_synonym(word,dictionary):
    s=get_synonyms(word)
    count=0
    for i in s:
        for key in dictionary:
            if i==key:
                count+=dictionary[key]
    return count
     

In [11]:
    # def linear_similarity_count(list1,list2):
#     count=0
#     total=min(len(list1),len(list2))
#     for i in range(0,total):
#         if list1[i]==list2[i]:
#             count+=1
#     return count
            

In [12]:
# def random_similarity_count(list1,list2,k):
#     count=0
#     total=min(len(list1),len(list2))
#     k=k%total
#     for i in range(0,total):
#         if list1[k]==list2[i]:
#             count+=1
#         k=(k+1)%total
#     return count

In [13]:
# def get_max_count(list1,list2):
#     counts=[]
#     limit=[3,5,7,10,60,88,44]
#     for i in limit:
#         counts.append(random_similarity_count(list1,list2,i))
#     counts.append(linear_similarity_count(list1,list2))
#     return max(counts)

###  Returns a dictionary containing frequencies of all the words in the given list (Count vectorizer)

In [87]:
def get_word_frequency(words_of_text):
    vectorizer = CountVectorizer()
    vectorizer.fit(words_of_text)
    word_frequency=vectorizer.vocabulary_
    return word_frequency

### Compares the two given dictionaries and returns a similarity percentage
(utility function for get_similarity_percentages () )

In [88]:
def find_similarity_percentage(d1,d2):
    words_count=list()
    for keyOne in d1:
        for keyTwo in d2:
             if keyOne == keyTwo:
#                 print(keyOne+" "+keyTwo)
                words_count.append((d2[keyTwo]))
                i=len(words_count)-1
                s=find_synonym(keyTwo,d2)
#                 print("s"+str(s))
                words_count[i]+=s
    total_words=len(d1)
#     print("total words :"+str(total_words)) 
    similar_words=len(words_count)
#     print("similar words :"+str(similar_words)) 
    percentage=(similar_words/total_words)*100
    return percentage

### Iterates over a dataframe and returns similarity percentage of each file against a test file

In [89]:
def get_similarity_percentages(test_file_name,df,word_count_dict):
    l=len(df)
    h=list(range(l))
    j=0
    for i in range(0,l):
        if df.iloc[i]['name']==test_file_name:
            j=i
            h[i]=None
    for i in range(0,l):
        if i!=j:
            h[i]=(find_similarity_percentage(df.iloc[j][word_count_dict],df.iloc[i][word_count_dict]))
    return h

### Implementation of count vectorizer

In [90]:
def own_count_vectorizer(f):
    diction={}
    my_list = list(set(f))
    for i in my_list:
        diction[i]=0
    for i in range(0,len(f)):
        if diction[f[i]]==0:
            for j in range(i,len(f)):
                if f[i]==f[j]:
                    diction[f[i]]=diction[f[i]]+1
    return diction

### Converts a given file into a list of words by using appropriate function
(utilility function for rows_to_store)

In [91]:
def get_file_text(name,extension):
    if extension=='.txt':
        return convert_text(name)
    elif extension=='.pdf':
        return convert_pdf(name)
    elif extension=='.docx':
        return convert_docx(name)

### Returns a dictionary containing names, extensions, text, words counts and plag boolean for a set of files

In [92]:
def rows_to_store(data):
    counter=0
    single_file=list()
    to_csv={}
    to_csv['serial']=[]
    to_csv['name']=[]
    to_csv['extension']=[]
    to_csv['text']=[]
    to_csv['word_count']=[]
    to_csv['plag']=[]
    for index, row in data.iterrows():
        to_csv['serial'].append(counter)
        to_csv['name'].append(row["name"])
        to_csv['extension'].append(row["extension"])
        temp_text=get_file_text(row["name"],row["extension"])
        to_csv['text'].append(temp_text)
        to_csv['word_count'].append(own_count_vectorizer(temp_text))
        to_csv['plag'].append(row['plag'])
        counter+=1
    return to_csv

### Reads a .txt file into a list 
(utility function for get_file_in_text)

In [93]:
def convert_text_to_para(filename):
    textfileobj=open(filename+'.txt','r',errors="ignore")
    text=textfileobj.read()
    return text    

### Reads a .pdf file into a list 
(utility function for get_file_in_text)

In [94]:
def convert_pdf_to_para(filename):
    pdffileobj=open(filename+'.pdf','rb')
    pdfreader=PyPDF2.PdfFileReader(pdffileobj)
    pageobj=pdfreader.getPage(0)
    text=pageobj.extractText()
    return text

### Reads a .docx file into a list 
(utility function for get_file_in_text)

In [95]:
def convert_docx_to_para(filename):
    text_list = ""
    doc = docx.Document(filename+".docx")
    for i in doc.paragraphs:
        text_list = text_list + i.text

    return text_list

### Reads a file into a list of sentences

In [96]:
def get_file_in_text(name,extension):
    if extension=='.txt':
        return convert_text_to_para(name)
    elif extension=='.pdf':
        return convert_pdf_to_para(name)
    elif extension=='.docx':
        return convert_docx_to_para(name)

### Convert training files into a dictionary of name, extension and plain text

In [97]:
def training_files_to_csv(folder_path):
    counter=0
    single_file=list()
    to_csv={}
    to_csv['serial']=[]
    to_csv['name']=[]
    to_csv['extension']=[]
    to_csv['text']=[]
    entries = os.listdir(folder_path)
    for file in entries:
        name,extension=os.path.splitext(folder_path+'/'+file)
        print(name)
        to_csv['serial'].append(counter)
        to_csv['name'].append(name)
        to_csv['extension'].append(extension)
        temp_text=get_file_in_text(name,extension)
        to_csv['text'].append(temp_text)
        counter+=1
    return to_csv

In [98]:
def testing_files_to_csv(folder_path):
    counter=0
    single_file=list()
    to_csv={}
    to_csv['serial']=[]
    to_csv['name']=[]
    to_csv['extension']=[]
    to_csv['text']=[]
    to_csv['word_count']=[]
    entries = os.listdir(folder_path)
    for file in entries:
        name,extension=os.path.splitext(folder_path+'/'+file)
        print(name)
        to_csv['serial'].append(counter)
        to_csv['name'].append(name)
        to_csv['extension'].append(extension)
        temp_text=get_file_text(name,extension)
        to_csv['text'].append(temp_text)
        to_csv['word_count'].append(own_count_vectorizer(temp_text))
        counter+=1
    return to_csv

# Data Collection

### Reading files from a folder and converting them into a dictionary of name, extension and plain text

In [99]:
data=training_files_to_csv('filess/files')

filess/files/20K-0234_Mannahil Miftah
filess/files/20K-1098(PF Assignment)
filess/files/20k-1610 final version
filess/files/20k-1610 rough work
filess/files/20k-1635 Rough writing of essay
filess/files/20k-1678 amna malik
filess/files/20K-1682_Ahsan_pre writing and first draft
filess/files/20K-1682_Muhammad Sarim Khan_Final draft (1)
filess/files/20K-1738 Final Draft Muhammad Abdaal
filess/files/20K-1739
filess/files/20k-1801 Essay(1)
filess/files/20k-1801 Essay
filess/files/20k-1801 Essay
filess/files/20k-1801 Rough draft
filess/files/20k-1801 Rough draft.pdf
filess/files/20K1057_Final Draft
filess/files/20K1057_RoughDraft
filess/files/20k1603 Muhammad Zaeem Aslam ECC Final Project Final Draft
filess/files/20k1603 Muhammad Zaeem Aslam ECC Final Project Rough Work 
filess/files/20K1637_FinalDraft
filess/files/20K1637_RoughDraft
filess/files/20K1669_ECC FINAL PROJECT(1)
filess/files/20K1669_ECC FINAL PROJECT(2)
filess/files/20K1669_ECC FINAL PROJECT
filess/files/20K1669_ECC FINAL PROJEC

### Converting that dictionary into a dataframe

In [100]:
data=pd.DataFrame(data)
data

,serial,name,extension,text
0,0,filess/files/20K-0234_Mannahil Miftah,.docx,TECHNOLOGY IS/IS NOT KILLING THE CREATIVITYBRA...
1,1,filess/files/20K-1098(PF Assignment),.docx,PROGRAMMING FUNDAMENTALS-ASSIGNMENT\nNested Lo...
2,2,filess/files/20k-1610 final version,.docx,“Technology is not killing the creativity.”Wha...
3,3,filess/files/20k-1610 rough work,.docx,“Technology is not killing the creativity.”BRA...
4,4,filess/files/20k-1635 Rough writing of essay,.docx,Rough writing of ess...
...,...,...,...,...
107,107,filess/files/Muhammad Farzan Ansari_20K-1636_E...,.pdf,HAS SOCIAL MEDIA IMPROVED HUMAN \nCOMMUNICATIO...
108,108,filess/files/muhammad shaheer_20k1087(1),.docx,ECC Final Project: Argumentative Essay1. Brain...
109,109,filess/files/muhammad shaheer_20k1087,.docx,ECC Final Project: Argumentative Essay1. Brain...
110,110,filess/files/Outline 20K-1803,.docx,Introductory ParagraphHook: General to specifi...


### Writing the dataframe into a csv file

In [101]:
data.to_csv('train_test_data.csv')

# Reading Mining

### Reading a csv file for training, testing into a dataframe

In [102]:
data  =  pd.read_csv("train_test_data.csv")

In [103]:
data

,Unnamed: 0,serial,name,extension,text
0,0,0,filess/files/20K-0234_Mannahil Miftah,.docx,TECHNOLOGY IS/IS NOT KILLING THE CREATIVITYBRA...
1,1,1,filess/files/20K-1098(PF Assignment),.docx,PROGRAMMING FUNDAMENTALS-ASSIGNMENT\nNested Lo...
2,2,2,filess/files/20k-1610 final version,.docx,“Technology is not killing the creativity.”Wha...
3,3,3,filess/files/20k-1610 rough work,.docx,“Technology is not killing the creativity.”BRA...
4,4,4,filess/files/20k-1635 Rough writing of essay,.docx,Rough writing of ess...
...,...,...,...,...,...
107,107,107,filess/files/Muhammad Farzan Ansari_20K-1636_E...,.pdf,HAS SOCIAL MEDIA IMPROVED HUMAN \nCOMMUNICATIO...
108,108,108,filess/files/muhammad shaheer_20k1087(1),.docx,ECC Final Project: Argumentative Essay1. Brain...
109,109,109,filess/files/muhammad shaheer_20k1087,.docx,ECC Final Project: Argumentative Essay1. Brain...
110,110,110,filess/files/Outline 20K-1803,.docx,Introductory ParagraphHook: General to specifi...


### Dropping unncessary columns

In [104]:
data = data.drop("Unnamed: 0" ,axis=1)

In [105]:
temp = [1,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,1,1,1,1,0,1,1,0,1,1,0,1,0,1,0,1,0,1,1,0,1,0,0,1,1,1,1,0,0,1,1,1,0,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,1,0,1,1,1,1,0,1,1,1,1,0,0,0,0,0,0,0,1,0,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1]


In [106]:
data["plag"]=temp
data

,serial,name,extension,text,plag
0,0,filess/files/20K-0234_Mannahil Miftah,.docx,TECHNOLOGY IS/IS NOT KILLING THE CREATIVITYBRA...,1
1,1,filess/files/20K-1098(PF Assignment),.docx,PROGRAMMING FUNDAMENTALS-ASSIGNMENT\nNested Lo...,0
2,2,filess/files/20k-1610 final version,.docx,“Technology is not killing the creativity.”Wha...,0
3,3,filess/files/20k-1610 rough work,.docx,“Technology is not killing the creativity.”BRA...,0
4,4,filess/files/20k-1635 Rough writing of essay,.docx,Rough writing of ess...,1
...,...,...,...,...,...
107,107,filess/files/Muhammad Farzan Ansari_20K-1636_E...,.pdf,HAS SOCIAL MEDIA IMPROVED HUMAN \nCOMMUNICATIO...,1
108,108,filess/files/muhammad shaheer_20k1087(1),.docx,ECC Final Project: Argumentative Essay1. Brain...,1
109,109,filess/files/muhammad shaheer_20k1087,.docx,ECC Final Project: Argumentative Essay1. Brain...,1
110,110,filess/files/Outline 20K-1803,.docx,Introductory ParagraphHook: General to specifi...,1


### Splitting the data into train and test 

In [107]:
from random import seed
from random import randrange

# Split a dataset into a train and test set
def train_test_split(dataset, split=0.60):
        train = list()
        train_size = split * len(dataset)
        dataset_copy = list(dataset)
        while len(train) < train_size:
                index = randrange(len(dataset_copy))
                train.append(dataset_copy.pop(index))
        return dataset, dataset_copy

# test train/test split
seed(1)
rows=data.values
train, test = train_test_split(rows)

In [35]:
# rows = data.values
# X = rows[:,0:4]
# y= rows[:,4]
# X_train, X_test, y_train, y_test = train_test_split(X, y)

# Model for training and prediction

In [108]:
class classifier:
    def __init__(self, neighbors):
        self.neighbours = neighbors
        self.train_data1=pd.DataFrame()
    def fit(self,data):
        s = rows_to_store(data)
        columns = ["serial","name","extension","text","word_count"]
        self.train_data1=pd.DataFrame(s,columns=columns)
        return s
    def predict(self,test_data):
        list1 = []
        for i in range(0,len(test_data)):
                b=get_similarity_percentages(test_data.iloc[i]["name"],self.train_data1,'word_count')
                self.train_data1['similarity_percentages']=b
                rows = self.train_data1[self.train_data1.similarity_percentages >= 50]
                if(rows.empty):
                    list1.append(0)
                else:
                    list1.append(1)
        return list1
    def accuracy_score(self,test_list,y):
        count=0;
        total=len(test_list)
        for i in range(0,len(test_list)):
            if(test_list[i] == y[i]):
                count=count+1
        return count/total * 100        
            

### Converting train, test data into separate dataframes

In [109]:
columns = ["serial","name","extension","text","plag"]
test_data = pd.DataFrame(test,columns=columns)
train_data = pd.DataFrame(train,columns=columns)

In [38]:
# temp=[]
# for i in range(0,22):
#     temp.append(1)
# for i in range(22,44):
#     temp.append(0)
# len(temp)
# test_data['plag']=temp
# test_data['plag']

In [110]:
test_data

,serial,name,extension,text,plag
0,1,filess/files/20K-1098(PF Assignment),.docx,PROGRAMMING FUNDAMENTALS-ASSIGNMENT\nNested Lo...,0
1,6,filess/files/20K-1682_Ahsan_pre writing and fi...,.docx,Muhammad Sarim Khan 20K-1682Topic:A government...,1
2,11,filess/files/20k-1801 Essay,.docx,TOPIC: A government has a responsibility to it...,1
3,12,filess/files/20k-1801 Essay,.pdf,20k\n-\n1801\n \nMujtaba Ahmed\n \n \nTOPIC:\n...,1
4,14,filess/files/20k-1801 Rough draft.pdf,.docx,TOPIC: A government has a responsibility to it...,1
5,15,filess/files/20K1057_Final Draft,.pdf,\nSONDOS SUHAIL 20K\n-\n10\n57\n \nGENETIC EN...,1
6,20,filess/files/20K1637_RoughDraft,.docx,BrainstormingTopic: Has ...,1
7,22,filess/files/20K1669_ECC FINAL PROJECT(2),.docx,ECC FINAL PROJECT(Argumrntative essay)Submitte...,1
8,23,filess/files/20K1669_ECC FINAL PROJECT,.docx,ECC FINAL PROJECT(Argumrntative essay)Submitte...,1
9,24,filess/files/20K1669_ECC FINAL PROJECT_document,.docx,ECC FINAL PROJECT(Argumrntative essay)Submitte...,1


### Fit training data into the classifier

In [111]:
columns = ["serial","name","extension","text","word_count","plag"]

to_classify = classifier(1)
check = to_classify.fit(train_data)
k = pd.DataFrame(check,columns=columns)
k

,serial,name,extension,text,word_count,plag
0,0,filess/files/20K-0234_Mannahil Miftah,.docx,"[technology, is/is, not, killing, the, creativ...","{'let': 2, 'long': 3, 'if': 3, 'development': ...",1
1,1,filess/files/20K-1098(PF Assignment),.docx,"[programming, fundamentals, assignment, nested...","{'n,': 1, 'if': 4, '#include': 2, 'i<=5': 1, '...",0
2,2,filess/files/20k-1610 final version,.docx,"[“technology, is, not, killing, the, creativit...","{'reclining': 1, 'long': 1, 'techs': 1, 'these...",0
3,3,filess/files/20k-1610 rough work,.docx,"[“technology, is, not, killing, the, creativit...","{'development': 1, 'paragraph': 2, 'techs': 1,...",0
4,4,filess/files/20k-1635 Rough writing of essay,.docx,"[rough, writing, of, essay, social, media, hel...","{'things': 2, 'get': 1, 'one': 5, 'learn': 3, ...",1
...,...,...,...,...,...,...
107,107,filess/files/Muhammad Farzan Ansari_20K-1636_E...,.pdf,HAS SOCIAL MEDIA IMPROVED HUMAN \nCOMMUNICATIO...,"{'k': 4, 'T': 7, 'N': 9, '5': 2, 'x': 2, 'j': ...",1
108,108,filess/files/muhammad shaheer_20k1087(1),.docx,"[ecc, final, project:, argumentative, essay, 1...","{'own': 1, 'if': 4, 'developing': 2, 'possible...",1
109,109,filess/files/muhammad shaheer_20k1087,.docx,"[ecc, final, project:, argumentative, essay, 1...","{'few': 1, 'set': 2, 'own': 2, 'if': 4, 'devel...",1
110,110,filess/files/Outline 20K-1803,.docx,"[introductory, paragraph, hook:, general, to, ...","{'privacy': 1, 'paragraph': 6, 'second': 1, 'o...",1


### Predict test data

In [112]:
# print(test_data.iloc[0]["name"])
predicted = to_classify.predict(test_data)
print(predicted)

[0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1]


### Predicted values into a varibale  

In [113]:
test_data_y = test_data['plag']

### Checking the accuracy of the classifier

In [114]:
print("The accuracy of the model is "+ str(to_classify.accuracy_score(predicted,test_data_y))+"% ")

The accuracy of the model is 86.36363636363636% 


# FOR FINDING FILES HAVING PLAG

In [115]:
check = get_similarity_percentages(test_data.iloc[1]["name"],k,'word_count')
k['similarity_percentages'] = check
rows = k[k.similarity_percentages >= 50]
# rows = rows.append(test_data.iloc[1])
rows


,serial,name,extension,text,word_count,plag,similarity_percentages
5,5,filess/files/20k-1678 amna malik,.docx,"[a, government, has, a, responsibility, to, it...","{'citizen': 2, 'these': 1, 'increase': 1, 'cat...",1,51.864407
7,7,filess/files/20K-1682_Muhammad Sarim Khan_Fina...,.docx,"[a, government, has, a, responsibility, to, it...","{'citizen': 2, 'these': 1, 'increase': 1, 'cat...",1,51.864407


# Running our Plagiarism Checker on a separate assignment folder

In [116]:
def check_plagiarism():
    folder=input("Enter Path of assignment folder")
    test_file=input("Enter file name you want to test against plagiarism")
    l=testing_files_to_csv(folder)
    df=pd.DataFrame(l)
    temp=get_similarity_percentages(test_file,df,"word_count")
    df['similarity_percentages']=temp
    p=[]
    for i in df['similarity_percentages']:
        if i>=20:
            p.append(1)
        else:
            p.append(0)
    df['plag']=p
    for i in range(0,len(df['plag'])):
        if df.iloc[i]['plag']==1:
            print("file name : "+df.iloc[i]['name']+"\n"+"Percentage : "+str(df.iloc[i]['similarity_percentages']))

In [117]:
check_plagiarism()

Enter Path of assignment foldertesting_folder
Enter file name you want to test against plagiarismassignment1
testing_folder/AI and our life
testing_folder/AI and us
testing_folder/AI and world
testing_folder/AI assignment
testing_folder/AI
testing_folder/Artifial Intelligence
testing_folder/Assignment 1
testing_folder/Assignment 2
testing_folder/How AI is affecting our lives
file name : testing_folder/AI and world
Percentage : 25.665399239543724
file name : testing_folder/Artifial Intelligence
Percentage : 26.045627376425855
file name : testing_folder/Assignment 1
Percentage : 26.045627376425855
